# FakeLenseV2 - Quick Start Tutorial

This notebook demonstrates the basic usage of FakeLenseV2 for fake news detection.

## Topics Covered
1. Loading the model
2. Making single predictions
3. Batch predictions
4. Understanding results

**Prerequisites**: Make sure you have trained a model or downloaded a pre-trained model.

## 1. Setup and Installation

In [ ]:
# Install dependencies (if not already installed)
# !pip install -e ../..

import sys
sys.path.append('../..')

import torch
from code.inference import InferenceEngine
from code.utils.config import get_default_config

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. Load the Model

The InferenceEngine loads the model once and reuses it for multiple predictions.

In [ ]:
# Path to your trained model
model_path = "../../models/best_model.pth"

# Load the inference engine
engine = InferenceEngine(model_path)

print("✅ Model loaded successfully!")

## 3. Single Prediction

Let's classify a single news article.

In [ ]:
# Example 1: Real news from a trusted source
article_1 = {
    "text": """Scientists at NASA announce discovery of Earth-like planet 
    in habitable zone. The planet, located in the TRAPPIST-1 system, 
    shows signs of potential water presence.""",
    "source": "Reuters",
    "social_reactions": 5000
}

prediction = engine.predict(
    text=article_1["text"],
    source=article_1["source"],
    social_reactions=article_1["social_reactions"]
)

# Map prediction to label
labels = {0: "Fake News", 1: "Suspicious News", 2: "Real News"}
print(f"\n📰 Article: {article_1['text'][:80]}...")
print(f"📍 Source: {article_1['source']}")
print(f"👥 Social Reactions: {article_1['social_reactions']:,}")
print(f"\n🔍 Prediction: {labels[prediction]} (Class {prediction})")

In [ ]:
# Example 2: Suspicious news from unknown source
article_2 = {
    "text": """Miracle cure for all diseases discovered by local doctor! 
    Pharmaceutical companies are trying to hide this revolutionary treatment. 
    Share before it's deleted!""",
    "source": "UnknownBlog",
    "social_reactions": 250000  # High viral spread
}

prediction = engine.predict(
    text=article_2["text"],
    source=article_2["source"],
    social_reactions=article_2["social_reactions"]
)

print(f"\n📰 Article: {article_2['text'][:80]}...")
print(f"📍 Source: {article_2['source']}")
print(f"👥 Social Reactions: {article_2['social_reactions']:,}")
print(f"\n🔍 Prediction: {labels[prediction]} (Class {prediction})")

## 4. Batch Predictions

Process multiple articles at once for better efficiency.

In [ ]:
# Multiple articles
articles = [
    {
        "text": "The federal government announces new regulations for AI ethics.",
        "source_reliability": "The New York Times",
        "social_reactions": 6200
    },
    {
        "text": "Aliens have landed in New York City, government confirms!",
        "source_reliability": "RandomSite",
        "social_reactions": 150000
    },
    {
        "text": "Stock market reaches record high amid economic recovery.",
        "source_reliability": "Bloomberg",
        "social_reactions": 8500
    }
]

# Batch prediction
predictions = engine.predict_batch(articles)

# Display results
print("\n" + "="*80)
print("BATCH PREDICTION RESULTS")
print("="*80)

for i, (article, pred) in enumerate(zip(articles, predictions), 1):
    print(f"\n[{i}] {article['text'][:60]}...")
    print(f"    Source: {article['source_reliability']}")
    print(f"    Prediction: {labels[pred]}")
    print(f"    {'-'*76}")

## 5. Understanding Source Reliability

The model uses pre-defined reliability scores for news sources.

In [ ]:
from code.utils.config import SOURCE_RELIABILITY_MAPPING
import pandas as pd

# Display source reliability scores
reliability_df = pd.DataFrame(
    list(SOURCE_RELIABILITY_MAPPING.items()),
    columns=["Source", "Reliability Score"]
).sort_values("Reliability Score", ascending=False)

print("\n📊 Source Reliability Scores:")
print(reliability_df.to_string(index=False))
print(f"\nDefault score for unknown sources: 0.50")

## 6. Analyzing Classification Patterns

Let's test how different factors affect predictions.

In [ ]:
# Same text, different sources
text = "Breaking: Major scientific breakthrough announced."

sources_to_test = ["Reuters", "CNN", "Fox News", "Unknown Blog"]
social_reactions = 10000

print("\n📊 Effect of Source Reliability:")
print(f"Text: '{text}'")
print(f"Social Reactions: {social_reactions:,}\n")

for source in sources_to_test:
    pred = engine.predict(text, source, social_reactions)
    reliability = SOURCE_RELIABILITY_MAPPING.get(source, 0.50)
    print(f"{source:20s} (score: {reliability:.2f}) → {labels[pred]}")

In [ ]:
# Same text and source, different social reactions
text = "Shocking revelation about celebrities!"
source = "HuffPost"

reactions_to_test = [100, 1000, 10000, 100000, 500000]

print("\n📊 Effect of Social Reactions:")
print(f"Text: '{text}'")
print(f"Source: {source}\n")

for reactions in reactions_to_test:
    pred = engine.predict(text, source, reactions)
    print(f"{reactions:>7,} reactions → {labels[pred]}")

## 7. Next Steps

- **Training Tutorial**: See `02_training_guide.ipynb` for custom training
- **Advanced Usage**: Check `03_advanced_features.ipynb`
- **API Integration**: See `README_USAGE.md` for REST API usage

## Resources

- [Documentation](../../README.md)
- [Usage Guide](../../README_USAGE.md)
- [Contributing](../../CONTRIBUTING.md)
- [GitHub Repository](https://github.com/Navy10021/FakeLenseV2)